In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib as mpl
from matplotlib import rc
rc('text',usetex=True)
rc('font',**{'family':'serif','serif':['Computer Modern Roman']})
import matplotlib.pyplot as plt
% matplotlib inline
% config InlineBackend.figure_format = 'svg'
% config InlineBackend.rc = {'figure.figsize': (5,3.5)}

In [5]:
import numpy as np
from numpy import sqrt,log
import numpy.linalg as la
import cvxpy as cvx
from helper import create_sample,max_error,in_error,out_error,solve
from cd.model.utility import LipschitzExpUtility as leu, RiskNeutralUtility as rnu
from cd.model.distrs import RademacherDistribution,UniformDistribution
import cd.model.synth_data as synth
import scaler
from itertools import product

In [11]:
p = 100
R = RademacherDistribution()
Xs = [RademacherDistribution() for _ in range(p)]

In [12]:
rm = 1
xi = sqrt(p)
lamb = 1

In [13]:
M = synth.GaussianMarket(Xs,R)
t_true = M.sample_t(100_000)
q_true = 1/lamb * t_true.mean(axis=0)

In [14]:
m = 1000
ns = np.arange(2,202,2)
ts = create_sample(t_true,n=max(ns),m=m)
errors = np.empty(len(ns))
inerrors = np.empty(len(ns))
for i,n in enumerate(ns):
    qs = solve(ts[:,:n],lamb=lamb)
    errors[i] = la.norm(qs-q_true,axis=1).max()
    inerrors[i] = in_error(qs,lamb=lamb).max()

In [16]:
tol = log(1/0.01)
plt.plot(ns,2*rm/lamb*sqrt(tol/2/ns))
plt.plot(ns,errors);
# plt.plot(ns,inerrors)

In [389]:
m = 5_000
ns = np.arange(2,502,2)
# ns = [2]
ts = create_sample(t_true,n=max(ns),m=m)
errors = np.zeros(len(ns))
outerrors = np.empty(len(ns))
inerrors = np.empty(len(ns))
maxinerrors = np.empty(len(ns))
hello = np.empty(len(ns))
for i,n in enumerate(ns):
    qs = solve(ts[:,:n],lamb=lamb)
    inerror = in_error(qs,lamb=lamb)
    outerror = out_error(qs,q_true=q_true,lamb=lamb)
    error = inerror - outerror
    idx = np.argmax(error)
    errors[i] = error[idx]
    outerrors[i] = outerror[idx]
    inerrors[i] = inerror[idx]
    maxinerrors[i] = la.norm(qs[idx]-q_true)
    hello[i] = la.norm(qs[idx])

In [393]:
tol = log(1/0.05)
qdist = 2*rm*xi/lamb*sqrt(tol/2/ns)
plt.plot(ns,hello)
plt.plot(ns,qdist+la.norm(q_true),'k--')
# plt.plot(ns,2*sqrt(tol/2/ns))
plt.plot(ns,maxinerrors)
plt.plot(ns,qdist,'k--')
# plt.plot(ns,hello*maxinerrors)


In [367]:
tol = log(1/0.01)
qdist = 2*rm*xi/lamb*sqrt(tol/2/ns)
plt.plot(ns,qdist**2 + xi/lamb*qdist)
plt.plot(ns,(2*xi/lamb)**2*sqrt(tol/2/ns)) # Borne generale
# plt.plot(ns,2*xi/lamb*sqrt(tol/2/ns)) # R independant de X --> ca fonctionne

plt.plot(ns,errors,label='Erreur totale')
# plt.plot(ns,inerrors,label='In Echantillon')
# plt.plot(ns,outerrors,label='Hors echantillon')
# plt.plot(ns,maxinerrors,label='Max In')
plt.title('Erreur de generalisation');
plt.legend();
# plt.axis(ymax=4)
# plt.plot(ns,0*ns+la.norm(q_true)**2)



In [178]:
q_true

array([ 1.0068,  0.9824])

In [20]:
m = 1000
ns = np.arange(2,202,2)
ts = create_sample(t_true,n=max(ns),m=m)
errors = np.empty(len(ns))
for i,n in enumerate(ns):
    qs = solve(ts[:,:n])
    error = out_error(qs,q_true=q_true)
    errors[i] = error.max()

In [24]:
plt.plot(ns,errors)
plt.title('Erreur hors echantillon');

In [185]:
m = 1000
ns = np.arange(2,202,2)
ts = create_sample(t_true,n=max(ns),m=m)
errors = np.empty(len(ns))
outerrors = np.empty_like(errors)
for i,n in enumerate(ns):
    qs = solve(ts[:,:n])
    error = in_error(qs)
    errors[i] = error.max()
    outerrors[i] = q_true@qs[np.argmax(error)]

In [186]:
plt.plot(ns,errors)
plt.plot(ns,outerrors)